In [1]:
import pandas as pd
import requests
import os
import zipfile

## 1.Extrair os dados da CVM

### 1.1 Diretório

In [3]:
# Sobe um nível no diretório
os.chdir("..")

In [4]:
# Verifica o novo diretório de trabalho atual
diretorio_atual = os.getcwd()
print(diretorio_atual)

C:\Workspace\Streamlit\api_contabil_TESTE


In [5]:
# Caminho dos diretorios
diretorio_preprocessed_I = os.path.join(diretorio_atual, "data_cvm\preprocessed_I")
diretorio_preprocessed_II = os.path.join(diretorio_atual, "data_cvm\preprocessed_II")
diretorio_processed = os.path.join(diretorio_atual, "data_cvm\processed")
diretorio_anual = os.path.join(diretorio_atual, "data_cvm\external\dados_cvm_anual")
diretorio_trimestral = os.path.join(diretorio_atual, "data_cvm\external\dados_cvm_trimestral")

In [6]:
# Criar os diretórios se não existirem
os.makedirs(diretorio_preprocessed_I, exist_ok=True)
os.makedirs(diretorio_preprocessed_II, exist_ok=True)
os.makedirs(diretorio_processed, exist_ok=True)
os.makedirs(diretorio_anual, exist_ok=True)
os.makedirs(diretorio_trimestral, exist_ok=True)

In [7]:
# Função para mudar o diretório
def mudar_diretorio(periodo):
    if periodo == "anual":
        os.chdir(diretorio_anual)
    elif periodo == "trimestral":
        os.chdir(diretorio_trimestral)
    else:
        raise ValueError("Período inválido. Escolha entre semestral ou trimestral.")

## 2. Download dos Arquivos

### 2.1 Anual

In [8]:
mudar_diretorio("anual")
anos = range(2011,2024)

url_base = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/"

for ano in anos:
    download = requests.get(url_base + f"dfp_cia_aberta_{ano}.zip")
    
    open(f"dfp_cia_aberta{ano}.zip", "wb").write(download.content)

## 2.2 Trimestral

In [9]:
mudar_diretorio("trimestral")
anos = range(2011,2024)

url_base = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/"

for ano in anos:
    download = requests.get(url_base + f"itr_cia_aberta_{ano}.zip")
    
    open(f"itr_cia_aberta_{ano}.zip", "wb").write(download.content)

# 3. Ler arquivos

In [10]:
# Função para ler os arquivos e gerar dataframe
def ler_arquivos_diretorio(diretorio):
    lista_demonstracoes = []
    for arquivo in os.listdir(diretorio):
        if arquivo == ".ipynb_checkpoints":
            pass
        else:
            arquivo_zip = zipfile.ZipFile(arquivo)        
            for planilha in arquivo_zip.namelist():                        
                demonstracao = pd.read_csv(arquivo_zip.open(planilha), sep=";", encoding = "ISO-8859-1", 
                                           dtype = {"ORDEM_EXERC": "category"})
                
                lista_demonstracoes.append(demonstracao)
    return pd.concat(lista_demonstracoes)   

## 3.1 Anual

In [11]:
mudar_diretorio("anual")
df_anual = ler_arquivos_diretorio(diretorio_anual)

## 3.2 Trimestral

In [12]:
mudar_diretorio("trimestral")
df_trimestral = ler_arquivos_diretorio(diretorio_trimestral)

# 4. Filtrar Dados da Empresa

## 4.1 Anual

In [13]:
df_anual_empresa = df_anual[df_anual['DENOM_CIA'] == 'EQUATORIAL ENERGIA S.A.'].copy()

## 4.2 Trimestral

In [14]:
df_trimestral_empresa = df_trimestral[df_trimestral['DENOM_CIA'] == 'EQUATORIAL ENERGIA S.A.'].copy()

# 5. Exportar Dados

## 5.1 Diretórios

In [15]:
# Diretorio atuaç
print(diretorio_atual)

C:\Workspace\Streamlit\api_contabil_TESTE


In [16]:
# Caminho dos diretorios
os.chdir(diretorio_preprocessed_I)
# Confirmar diretorio
diretorio_atual = os.getcwd()
print(diretorio_atual)

C:\Workspace\Streamlit\api_contabil_TESTE\data_cvm\preprocessed_I


In [17]:
df_anual_empresa.to_csv('equatorial_anual.csv', index=False)

In [18]:
df_trimestral_empresa.to_csv('equatorial_trimestral.csv', index=False)
